In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Import necessary libraries
import os
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf


In [12]:
def dice_coefficient(y_true, y_pred, smooth=1.0):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(tf.keras.backend.reshape(y_true_f, [-1]) * tf.keras.backend.reshape(y_pred_f, [-1]))
    return (2. * intersection + smooth) / (tf.keras.backend.sum(tf.keras.backend.reshape(y_true_f, [-1])) + tf.keras.backend.sum(tf.keras.backend.reshape(y_pred_f, [-1])) + smooth)

def dice_coefficient_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.math.sigmoid(y_pred)
    numerator = 2.0 * tf.reduce_sum(tf.multiply(y_true, y_pred), axis=(1, 2, 3))

    #numerator = 2.0 * tf.reduce_sum(y_true * y_pred, axis=(1, 2, 3))
    denominator = tf.reduce_sum(tf.square(y_true) + tf.square(y_pred), axis=(1, 2, 3))

    tf.print('y_true shape:', tf.shape(y_true))
    tf.print('y_pred shape:', tf.shape(y_pred))
    tf.print('numerator shape:', tf.shape(numerator))
    tf.print('denominator shape:', tf.shape(denominator))

    return tf.reduce_mean(1.0 - numerator / denominator)


In [13]:
from tensorflow.keras.layers import Resizing

def CardioSegNet(input_shape=(256, 256, 1)):
    inputs = Input(input_shape)
    
    # Resize input images
    resized_inputs = Resizing(224, 224, interpolation='bilinear')(inputs)

    # Encoder: MobileNetV2
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet', input_tensor=resized_inputs)
    base_model.trainable = False


    block1 = base_model.get_layer('block_1_expand_relu').output
    block2 = base_model.get_layer('block_3_expand_relu').output
    block3 = base_model.get_layer('block_6_expand_relu').output
    block4 = base_model.get_layer('block_13_expand_relu').output
    block5 = base_model.get_layer('block_16_project').output

    # Decoder
    x = UpSampling2D((2, 2))(block5)
    x = Concatenate(axis=-1)([x, block4])
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)

    x = UpSampling2D((2, 2))(x)
    x = Concatenate(axis=-1)([x, block3])
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)

    x = UpSampling2D((2, 2))(x)
    x = Concatenate(axis=-1)([x, block2])
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)

    x = UpSampling2D((2, 2))(x)
    x = Concatenate(axis=-1)([x, block1])
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)

    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)

    model = Model(inputs, outputs)
    return model




In [14]:
# Load preprocessed data
def load_preprocessed_data(path):
    X = []
    y = []
    for file in sorted(os.listdir(path)):
        if file.endswith('.npz'):
            data = np.load(os.path.join(path, file))
            X.append(data['X'])
            y.append(data['y'])
    X = np.concatenate(X)
    y = np.concatenate(y)
    return X, y

In [15]:
preprocessed_data_path = "/content/drive/MyDrive/CAM/CAM1/preprocessed_data/"
X, y = load_preprocessed_data(preprocessed_data_path)
# Move the channel axis to the end
X = np.moveaxis(X, 1, -1)
y = np.moveaxis(y, 1, -1)

# Repeat the single channel three times to create a 3-channel image
X = np.repeat(X, 3, axis=-1)

In [16]:
from sklearn.model_selection import train_test_split
# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_val = np.squeeze(y_val, axis=-1)


In [17]:
# Data augmentation
data_gen_args = dict(rotation_range=15,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     shear_range=0.1,
                     zoom_range=0.1,
                     horizontal_flip=True,
                     fill_mode='nearest')

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 1
image_datagen.fit(X_train, augment=True, seed=seed)
mask_datagen.fit(y_train, augment=True, seed=seed)

image_generator = image_datagen.flow(X_train, seed=seed, batch_size=8)
mask_generator = mask_datagen.flow(y_train, seed=seed, batch_size=8)

train_generator = zip(image_generator, mask_generator)

In [18]:
# Create and compile the model
model = CardioSegNet(input_shape=(256, 256, 3))
model.compile(optimizer=Adam(learning_rate=1e-4), loss=dice_coefficient_loss, metrics=[dice_coefficient])

In [19]:
# Train the model
#model_checkpoint = ModelCheckpoint('CardioSegNet.h5', monitor='val_dice_coefficient', mode='max', save_best_only=True)
#from tensorflow.keras.callbacks import EarlyStopping

# Train the model
model_checkpoint = ModelCheckpoint('CardioSegNet.h5', monitor='val_dice_coefficient', mode='max', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_dice_coefficient', mode='max', patience=10, restore_best_weights=True)

history = model.fit(train_generator, steps_per_epoch= 8, epochs=20, validation_data=(X_val, y_val), callbacks=[model_checkpoint, early_stopping])


Epoch 1/20
y_true shape: [8 256 256 1]


InvalidArgumentError: ignored

In [ ]:


#history = model.fit(train_generator, steps_per_epoch=len(X_train) // 8, epochs=100, validation_data=(X_val, y_val), callbacks=[model_checkpoint])

# Visualize the results
import matplotlib.pyplot as plt

def plot_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['dice_coefficient'])
    plt.plot(history.history['val_dice_coefficient'])
    plt.title('Dice Coefficient')
    plt.xlabel('Epoch')
    plt.ylabel('Dice Coefficient')
    plt.legend(['Train', 'Validation'])

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'])
    plt.show()

plot_history(history)

# Load the best model
model.load_weights('cardiosegnet.h5')

# Make predictions
predictions = model.predict(X_val)

# Visualize predictions
def visualize_predictions(X_val, y_val, predictions, num_samples=5):
    indices = np.random.choice(len(X_val), num_samples, replace=False)
        for i, idx in enumerate(indices):
        plt.subplot(num_samples, 3, i * 3 + 1)
        plt.imshow(X_val[idx].reshape(256, 256), cmap='gray')
        plt.title('Input Image')
        plt.axis('off')

        plt.subplot(num_samples, 3, i * 3 + 2)
        plt.imshow(y_val[idx].reshape(256, 256), cmap='gray')
        plt.title('Ground Truth Mask')
        plt.axis('off')

        plt.subplot(num_samples, 3, i * 3 + 3)
        plt.imshow(predictions[idx].reshape(256, 256), cmap='gray')
        plt.title('Predicted Mask')
        plt.axis('off')
    plt.show()

visualize_predictions(X_val, y_val, predictions, num_samples=5)



In [13]:
from keras.models import Model

# Assuming that your model is called 'model'
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 resizing (Resizing)            (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['resizing[0][0]']               
                                )                                                                 
                                                                                              

In [11]:
import numpy as np

# Assuming your X_val and y_val arrays are numpy arrays
print("Shape of X_val:", np.shape(X_val))
print("Shape of y_val:", np.shape(y_val))


Shape of X_val: (180, 256, 256, 3)
Shape of y_val: (180, 256, 256, 1)
